In [1]:
import sys
sys.path.append('/Users/alejandromeridamaroto/aily/vcs/aily-ai-sanofi/src/lab_capa/src')

In [2]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

data = load_breast_cancer()

X = pd.DataFrame(data.data)

y = data.target

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       569 non-null    float64
 1   1       569 non-null    float64
 2   2       569 non-null    float64
 3   3       569 non-null    float64
 4   4       569 non-null    float64
 5   5       569 non-null    float64
 6   6       569 non-null    float64
 7   7       569 non-null    float64
 8   8       569 non-null    float64
 9   9       569 non-null    float64
 10  10      569 non-null    float64
 11  11      569 non-null    float64
 12  12      569 non-null    float64
 13  13      569 non-null    float64
 14  14      569 non-null    float64
 15  15      569 non-null    float64
 16  16      569 non-null    float64
 17  17      569 non-null    float64
 18  18      569 non-null    float64
 19  19      569 non-null    float64
 20  20      569 non-null    float64
 21  21      569 non-null    float64
 22  22

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

x_train

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
269,10.710,20.39,69.50,344.9,0.10820,0.12890,0.084480,0.02867,0.1668,0.06862,...,11.69,25.21,76.51,410.4,0.1335,0.2550,0.25340,0.08600,0.2605,0.08701
103,9.876,19.40,63.95,298.3,0.10050,0.09697,0.061540,0.03029,0.1945,0.06322,...,10.76,26.83,72.22,361.2,0.1559,0.2302,0.26440,0.09749,0.2622,0.08490
152,9.731,15.34,63.78,300.2,0.10720,0.15990,0.410800,0.07857,0.2548,0.09296,...,11.02,19.49,71.04,380.5,0.1292,0.2772,0.82160,0.15710,0.3108,0.12590
348,11.470,16.03,73.02,402.7,0.09076,0.05886,0.025870,0.02322,0.1634,0.06372,...,12.51,20.79,79.67,475.8,0.1531,0.1120,0.09823,0.06548,0.2851,0.08763
520,9.295,13.90,59.96,257.8,0.13710,0.12250,0.033320,0.02421,0.2197,0.07696,...,10.57,17.84,67.84,326.6,0.1850,0.2097,0.09996,0.07262,0.3681,0.08982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,14.470,24.99,95.81,656.4,0.08837,0.12300,0.100900,0.03890,0.1872,0.06341,...,16.22,31.73,113.50,808.9,0.1340,0.4202,0.40400,0.12050,0.3187,0.10230
371,15.190,13.21,97.65,711.8,0.07963,0.06934,0.033930,0.02657,0.1721,0.05544,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766
259,15.530,33.56,103.70,744.9,0.10630,0.16390,0.175100,0.08399,0.2091,0.06650,...,18.49,49.54,126.30,1035.0,0.1883,0.5564,0.57030,0.20140,0.3512,0.12040
556,10.160,19.59,64.73,311.7,0.10030,0.07504,0.005025,0.01116,0.1791,0.06331,...,10.65,22.88,67.88,347.3,0.1265,0.1200,0.01005,0.02232,0.2262,0.06742


In [18]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
import optuna 
from dvclive.xgb import DVCLiveCallback
from dvclive import Live

from models.classification.tuner import OptunaModelOptimizer


N_TRIALS = 30
SCORING = roc_auc_score
SEED = 42

tracking_metrics = {
    "accuracy": accuracy_score,
    "f1": f1_score,
    "recall": recall_score,
}

classifier = xgb.XGBClassifier

param_grid = {
    'n_estimators': optuna.distributions.IntDistribution(50, 200),
    'max_depth': optuna.distributions.IntDistribution(5, 40),
    'learning_rate': optuna.distributions.FloatDistribution(0.01, 1),
}


fit_params = {
    'eval_set':[(x_test, y_test)],
    'verbose': False,
            }


tracking_metrics_params = {
                        "f1": {
                            "average":"binary"
                        },
                        "recall": {
                            "average":"binary"
                        },
                    }



with Live(str("results"), save_dvc_exp=True, report="notebook") as live:     
    fixed_param_grid = {
    'objective': 'binary:logistic',
    'verbosity': 0,
    'early_stopping_rounds': 2,
    'callbacks':[DVCLiveCallback()]
}
    optimizer = OptunaModelOptimizer(
        classifier=classifier,
        scoring_metric=SCORING,
        param_grid=param_grid,
        fit_params=fit_params,
        fixed_param_grid=fixed_param_grid,
        tracking_metrics=tracking_metrics,
        tracking_metrics_params=tracking_metrics_params,
        cv_folds=4,
        seed=SEED
    )

optimizer.fit(x_train, y_train, x_test, y_test, n_trials=N_TRIALS, calculate_metrics=True)


Add a commit (`git commit`) to save experiments.
Add a commit (`git commit`) to save experiments.


# DVC Report

metrics.json




Add a commit (`git commit`) to save experiments.
Add a commit (`git commit`) to save experiments.
[I 2023-11-20 17:07:56,011] A new study created in memory with name: no-name-41f88023-8893-4aac-8aa1-e2051d333723
[W 2023-11-20 17:07:56,014] Trial 0 failed with parameters: {'n_estimators': 94, 'max_depth': 21, 'learning_rate': 0.32862629464594134} because of the following error: TypeError("'XGBClassifier' object is not callable").
Traceback (most recent call last):
  File "/Users/alejandromeridamaroto/.pyenv/versions/3.10.6/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/Users/alejandromeridamaroto/aily/vcs/aily-ai-sanofi/src/lab_capa/src/models/classification/tuner.py", line 313, in <lambda>
    objective_with_data = lambda trial: self._objective(
  File "/Users/alejandromeridamaroto/aily/vcs/aily-ai-sanofi/src/lab_capa/src/models/classification/tuner.py", line 224, in _objective
    clf = self.classifier(**para

TypeError: 'XGBClassifier' object is not callable

In [ ]:
from dvclive import Live

with Live("eval", dvcyaml=False) as live:
    if not live.summary:
        live.summary = {"roc_auc": {}, "f1-score": {}, "accuracy": {}, "recall": {}}
    for row, value in df_result.iterrows():
        live.summary["roc_auc"][row] = value['user_attrs_scoring_metric_value']
        live.summary["f1-score"][row] = value['user_attrs_test_f1']
        live.summary["accuracy"][row] = value['user_attrs_test_accuracy']
        live.summary["recall"][row] = value['user_attrs_test_recall']

live.summary